<a href="https://colab.research.google.com/github/myozaw5288/git/blob/master/dz1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
data=pd.read_csv('weatherAUS.csv', sep=",")
data.shape


(145460, 23)

In [2]:
data.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

In [3]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    data, data['WindGustSpeed'], test_size= 0.3, random_state= 1)
# Размер обучающей выборки
X_train.shape, y_train.shape


((101822, 23), (101822,))

In [7]:
# Размер тестовой выборки
X_test.shape, y_test.shape


((43638, 23), (43638,))

In [8]:
wea_ds=tf.read=tf.data.experimental.make_csv_dataset(file_pattern='weatherAUS.csv',
                                                     batch_size=25, label_name='WindGustSpeed', num_epochs=1)

In [9]:
wea_ds

<PrefetchDataset shapes: (OrderedDict([(Date, (None,)), (Location, (None,)), (MinTemp, (None,)), (MaxTemp, (None,)), (Rainfall, (None,)), (Evaporation, (None,)), (Sunshine, (None,)), (WindGustDir, (None,)), (WindDir9am, (None,)), (WindDir3pm, (None,)), (WindSpeed9am, (None,)), (WindSpeed3pm, (None,)), (Humidity9am, (None,)), (Humidity3pm, (None,)), (Pressure9am, (None,)), (Pressure3pm, (None,)), (Cloud9am, (None,)), (Cloud3pm, (None,)), (Temp9am, (None,)), (Temp3pm, (None,)), (RainToday, (None,)), (RainTomorrow, (None,))]), (None,)), types: (OrderedDict([(Date, tf.string), (Location, tf.string), (MinTemp, tf.float32), (MaxTemp, tf.float32), (Rainfall, tf.string), (Evaporation, tf.string), (Sunshine, tf.string), (WindGustDir, tf.string), (WindDir9am, tf.string), (WindDir3pm, tf.string), (WindSpeed9am, tf.string), (WindSpeed3pm, tf.string), (Humidity9am, tf.int32), (Humidity3pm, tf.int32), (Pressure9am, tf.float32), (Pressure3pm, tf.float32), (Cloud9am, tf.string), (Cloud3pm, tf.string),